In [90]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
from datetime import datetime

### datos necesarios para el dataframe:

- web scrapping: price, product_name, product_id, product_brand
- url
- fecha del dia

--> el dataframe solo podrá tener una entrada por dia y por producto (poner consiciones de si ya existe la fecha de hoy para ese nombre de producto no añadir nada)


--> columnas (url, brand, name, id, price)


### output a) un único DF:

Un dataframe con las siguientes caracteristicas:

- el dataframe solo podrá tener una entrada por cada producto al día (poner condiciones de si ya existe la fecha de hoy para ese nombre de producto no añadir nada)

- columnas (date_hyphen, date_slash, date_number, url, brand, name, id, price)

### output B) un DF por cada producto:

De esta manera cada dia sería una única entrada por cada DF

### test con la silla cedric

In [4]:
url_cedric = 'https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cedric/'

In [5]:
response_cedric = requests.get(url_cedric)
html_cedric = response_cedric.content
parsed_content_cedric = bs4.BeautifulSoup(html_cedric, "html.parser")

In [40]:
price_cedric_class = parsed_content_cedric.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0]

In [81]:
#final_price_cedric = price_cedric_class.text.replace("€","").strip().replace(",",".")
final_price_cedric = re.sub("[^\d|\,]","",price_cedric_class.text).replace(",",".")
final_price_cedric

'252.74'

In [86]:
final_name_cedric = str(parsed_content_cedric).split("ficha_product_name='")[1].split("';")[0].strip()
final_name_cedric

'Cedric B07SRYBJ6L'

In [87]:
final_id_cedric = str(parsed_content_cedric).split("ficha_product_id='")[1].split("';")[0].strip()
final_id_cedric

'B07SRYBJ6L'

In [88]:
final_brand_cedric = str(parsed_content_cedric).split("ficha_product_brand='")[1].split("';")[0].strip()
final_brand_cedric

'Cedric'

In [118]:
date_hyphen = datetime.today().strftime('%Y-%m-%d')
date_hyphen

'2022-04-12'

In [106]:
date_slash = datetime.today().strftime('%Y/%m/%d')
date_slash

'2022/04/12'

In [114]:
date_number = int(datetime.today().strftime('%Y%m%d'))
date_number

20220412

### test con un conjunto de urls

In [123]:
urls_products_list = [
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cedric/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/tlv-myx-801-1/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn55bk/"
]
#falta añadir condición a price de que si no existe pongamos un 0"

In [138]:
parsed_products_content_list = [bs4.BeautifulSoup(requests.get(i).content, "html.parser") for i in urls_products_list]
parsed_products_class = [i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text for i in parsed_products_content_list]
final_price_products = [re.sub("[^\d|\,]","",i).replace(",",".") for i in parsed_products_class]
final_name_products = [str(i).split("ficha_product_name='")[1].split("';")[0].strip() for i in parsed_products_content_list]
final_id_products = [str(i).split("ficha_product_id='")[1].split("';")[0].strip() for i in parsed_products_content_list]
final_brand_products = [str(i).split("ficha_product_brand='")[1].split("';")[0].strip() for i in parsed_products_content_list]
final_date_hyphen_products = [datetime.today().strftime('%Y-%m-%d') for i in range(len(urls_products_list))]
final_date_slash_products = [datetime.today().strftime('%Y/%m/%d') for i in range(len(urls_products_list))]
final_date_number_products = [int(datetime.today().strftime('%Y%m%d')) for i in range(len(urls_products_list))]

### test cuando no hay stock

In [153]:
url_no_price = "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/silla-gamer-bgeu-a136-sencillez-y-buen-precio-ofertas-2021/"

In [149]:
response_np = requests.get(url_no_price)
html_np = response_np.content
parsed_content_np = bs4.BeautifulSoup(html_np, "html.parser")
price_cedric_np = parsed_content_np.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text

In [155]:
price_cedric_np

''

In [151]:
#si está sin stock aparecerá "" --> ponerle condicion para que cuando pase sea un "0"

In [156]:
if price_cedric_np == "":
    print(0)

0


In [157]:
#hay que pasarlo al list comprenhension (una opcion sería que cuando esté el dataframe ponerle un apply(función para susitituirlo))

### test cuando no hay stock


In [152]:
url_wrong = "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/femor/"

### meter tambien una condicion de else